In [1]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# import yfinance as yf
from scipy.signal import argrelextrema
# from collections import deque
# from matplotlib.lines import Line2D
import pandas_ta as ta
import ccxt
from datetime import datetime, timedelta
# import plotly.graph_objects as go
# import inspect 
import os
# import ccxt_data_fetch
from IPython.display import display, HTML
from tqdm import tqdm

In [2]:
import os
import plotly.graph_objects as go

dataframes = {}

list_of=[]
timeframe='1h'
# dirr = f"C:/Users/Grant/Desktop/work_git/work/data/2022-01-28_12/{timeframe}"
outdir=f"{os.getcwd()}/data/{timeframe}"


for filename in os.listdir(outdir):
    symbol = filename.replace(f"_{timeframe}.csv","")
    df = pd.read_csv(f"{outdir}/{filename}", index_col='Time')
    # if len(df) > 17:

# for filename in os.listdir('datasets'):
    #print(filename)
    # symbol = filename.split(".")[0]
    #print(symbol)
    # df = pandas.read_csv('datasets/{}'.format(filename))
    if df.empty:
        continue

    df['20sma'] = df['Close'].rolling(window=20).mean()
    df['stddev'] = df['Close'].rolling(window=20).std()
    df['lower_band'] = df['20sma'] - (2 * df['stddev'])
    df['upper_band'] = df['20sma'] + (2 * df['stddev'])

    df['TR'] = abs(df['High'] - df['Low'])
    df['ATR'] = df['TR'].rolling(window=20).mean()

    df['lower_keltner'] = df['20sma'] - (df['ATR'] * 1.5)
    df['upper_keltner'] = df['20sma'] + (df['ATR'] * 1.5)

    def in_squeeze(df):
        return df['lower_band'] > df['lower_keltner'] and df['upper_band'] < df['upper_keltner']

    df['squeeze_on'] = df.apply(in_squeeze, axis=1)

    if df.iloc[-3]['squeeze_on'] and not df.iloc[-1]['squeeze_on']:
        print("{} is coming out the squeeze".format(symbol))

    # save all dataframes to a dictionary
    # we can chart individual names below by calling the chart() function
    dataframes[symbol] = df


def chart(df):
    candlestick = go.Candlestick(x=df.index, open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'])
    upper_band = go.Scatter(x=df.index, y=df['upper_band'], name='Upper Bollinger Band', line={'color': 'red'})
    lower_band = go.Scatter(x=df.index, y=df['lower_band'], name='Lower Bollinger Band', line={'color': 'red'})

    upper_keltner = go.Scatter(x=df.index, y=df['upper_keltner'], name='Upper Keltner Channel', line={'color': 'blue'})
    lower_keltner = go.Scatter(x=df.index, y=df['lower_keltner'], name='Lower Keltner Channel', line={'color': 'blue'})

    fig = go.Figure(data=[candlestick, upper_band, lower_band, upper_keltner, lower_keltner])
    # fig.layout.xaxis.type = 'category'
    fig.layout.xaxis.rangeslider.visible = False
    fig.show()



FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:/Users/Grant/Desktop/work_git/work/data/2022-01-28_12/4h'

In [ ]:
df = dataframes['ARUSDT']
chart(df)